# Single Node Single GPU Training

In [1]:
import numpy as np, pandas as pd
import requests, io, os, datetime, re
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.applications import resnet50
from keras.preprocessing import image
import dask
import time

2021-09-24 20:29:22.885809: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
%pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [3]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: skirmer (use `wandb login --relogin` to force relogin)


True

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2021-09-24 20:29:26.938463: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-24 20:29:26.939361: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-24 20:29:26.983589: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-24 20:29:26.984190: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-09-24 20:29:26.984228: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-09-24 20:29:26.985931: I tensorflow/stream_executor/platform/default/dso_loade

In [5]:
def train_model_fit(n_epochs, base_lr, batchsize, classes):
    
    model = tf.keras.applications.ResNet50(
        include_top=True,
        weights=None,
        classes=classes)
        
    # --------- Start wandb --------- #
    wandb.init(entity=[YOURUSERNAMEHERE], project=[YOURPROJECTNAMEHERE], config=wbargs)    
    
    # Data
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        'datasets/birds/train',
        image_size=(224,224),
        batch_size=batchsize
    ).prefetch(2).cache().shuffle(1000)
        
    valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
        'datasets/birds/valid',
        image_size=(224,224),
        batch_size=batchsize
    ).prefetch(2)
   
    optimizer = keras.optimizers.Adam(lr=base_lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    start = time.time()

    model.fit(
        train_ds, 
        epochs=n_epochs, 
        validation_data=valid_ds,
        callbacks=[WandbCallback()]
    )
    end = time.time()-start
    print("model training time", end)
    wandb.log({"training_time":end})

    # Close your wandb run 
    wandb.run.finish()
    
    tf.keras.models.save_model(model, 'model/keras_single/')

In [6]:
model_params = {'n_epochs': 50, 
                'base_lr': .02,
                'classes': 285,
               'batchsize': 64}

wbargs = {**model_params,
    'Notes':"tf_v100_2x",
    'Tags': ['single', 'gpu', 'tensorflow'],
    'dataset':"Birds",
    'architecture':"ResNet50"}

In [ ]:
tester = train_model_fit(**model_params)

2021-09-24 20:29:29.314560: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Found 43622 files belonging to 300 classes.
Found 1500 files belonging to 300 classes.
Epoch 1/50


2021-09-24 20:29:38.669072: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-24 20:29:38.723150: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2499995000 Hz
2021-09-24 20:29:39.882837: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-09-24 20:29:49.908934: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 116 of 1000
2021-09-24 20:29:59.904395: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 206 of 1000
2021-09-24 20:30:10.023425: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 274 of 1000
2021-09-24 20:30:19.887107: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 

In [ ]:
import os
os.getcwd()